In [1]:
import numpy as np


In [2]:
def get_t(feat_range, bin_size, offset):
    tiling = []
    accumulate = feat_range[0] + offset + bin_size
    tiling.append(feat_range[0] + offset)
    while(accumulate < feat_range[1]):
        tiling.append(accumulate)
        accumulate += (bin_size + 1)
    return np.array(tiling)

In [3]:
def clip_val(x, ub):
    lb = 0
    if x < lb:
        return lb
    if x > ub:
        return ub
    return x

In [10]:
def get_tilings():
    #dealer_showing: [1,4][4,7][7,10]
    feat_range = [1,10]
    bin_size = 3
    offset = 0
    a12 = get_t(feat_range, bin_size, offset)

    #player_sum : [1,6][7,12][13,18]
    feat_range = [1,21]
    bin_size = 5
    offset = 0
    a3 = get_t(feat_range, bin_size, offset)

    #player_sum : [4,9][10,15][16,21]
    feat_range = [1,21]
    bin_size = 5
    offset = 3
    a4 = get_t(feat_range, bin_size, offset)

    #actions : [hit, stick]
    a5 = np.array([0,1])
    
    tilings = a12, a3, a4, a5
    return tilings

In [26]:
def get_coding(state, action, tilings):
    #matrix to store the tile coded state:
    t_coding = np.zeros((3,6,2))
    
    #unpack tilings : 
    a12, a3, a4, a5 = tilings
    
    #upper bounds for index clipping
    ub_0 = t_coding.shape[0] - 1
    ub_1 = t_coding.shape[1] - 1
    ub_2 = t_coding.shape[2] - 1
    
    #tile coding:
    #axis_x store the values of which indices of the t_coding matrix to make 1.
    #can be multiple along a tiling because of overlapping tiles
    axis_1 = []
    axis_0 = []
    
    axis_2 = np.digitize(action, a5, right = True)
    
    #multiplied by 2 to get an alternating effect of indicing
    #the a3 tiling will append even indices, and the a4 one will append odd indices
    axis_1.append(clip_val(2 * (np.digitize(state[0],a3, right = True) - 1), ub_1))
    axis_1.append(2 * (np.digitize(state[0],a4) - 1) + 1)
    axis_1 = np.array(axis_1)
    
    axis_0.append(clip_val(np.digitize(state[1], a12, right = False) - 1, ub_0))
    axis_0.append(clip_val(np.digitize(state[1], a12, right = True) - 1, ub_0))
    axis_0 = np.array(axis_0)
    #remove the -1 index that comes for state[1] = 1 in interval [4,9]
    axis_1 = axis_1[axis_1 >= 0]
    print(axis_0,axis_1,axis_2)
    t_coding[axis_0,axis_1,axis_2] = 1

    return np.reshape(t_coding,(-1))

In [31]:
tilings = get_tilings()
state = (5,1)
action = 1
print(get_coding(state, action, tilings))

[0 0] [0 1] 1
[0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
